# Beam stiffness matrix

In this notebook 

In [119]:
from sympy import *
import matplotlib.pyplot as plt

In [120]:
xi = symbols('xi')

In [121]:
N1 = xi**3/4 - 3*xi/4 + 1/2
N2 = -xi**3/4 + xi**2/4 + xi/4 - 1/4
N3 = -xi**3/4 + 3*xi/4 + 1/2
N4 = -xi**3/4 - xi**2/4 + xi/4 + 1/4

The bending moment is given as $M(\xi) = - EI \partial^2 w(\xi)/\partial x^2$. With the curvature-displacement matrix $B$ and a vector of the degrees of freedom, $W$, where $W_1=w_1$ (deflection at the left hand side, i.e. at $\xi=-1$), $W_2=\theta_1$ (rotation at the left hand side), 
$W_3=w_2$ (deflection at the right hand side, i.e. at $\xi=+1$), $W_4=\theta_2$ (rotation at the right hand side), we can write

$$
M(\xi) = - EI B W
$$

Using `sympy` we write

In [122]:
W_1, W_2, W_3, W_4 = symbols('W_1, W_2, W_3, W_4')
W = Matrix([[W_1], [W_2], [W_3], [W_4]])
E, I, h = symbols('E, I, h')

The curvature-displacement matrix $B$ is constructed from the second derivatives of the basis functions, 

In [123]:
d2N1x2 = diff(N1, xi, 2) * (2/h)**2
d2N2x2 = diff(N2, xi, 2) * (2/h)**2
d2N3x2 = diff(N3, xi, 2) * (2/h)**2
d2N4x2 = diff(N4, xi, 2) * (2/h)**2

as

In [124]:
B = Matrix([d2N1x2, (h/2) * d2N2x2, d2N3x2, (h/2) * d2N4x2]).reshape(1, 4)
print(B)

Matrix([[6*xi/h**2, (1 - 3*xi)/h, -6*xi/h**2, -(3*xi + 1)/h]])


So now we can define symbolically an expression for the bending moment:

In [125]:
M = - E * I * (B * W)[0]
print('M = ', M)


M =  -E*I*(6*W_1*xi/h**2 + W_2*(1 - 3*xi)/h - 6*W_3*xi/h**2 - W_4*(3*xi + 1)/h)


Note that the bending moment along the beam varies linearly as a function of $\xi$, and depends linearly on $W_1, ..., W_4$. 

We are looking for the stiffness matrix of the beam, namely  the matrix $K$ that gives the forces acting on the endpoints of the beam $F$ in terms of the displacements $W$ (well, generalized displacements, really, since they include translations and rotations). So

$$
F = K \times W
$$

We can obtain the entries of the stiffness matrix using Castgliano's theorem. First we express the strain energy stored in the beam as

$$
U  = (1/2) \int_0^h M^2/(EI) dx = 1/(2EI) \int_{-1}^{+1} M^2 d\xi (h/2) = h/(4EI)\int_{-1}^{+1} M^2 d\xi
$$

where we can evaluate the necessary integral as

In [126]:
U = h/(4*E*I) * integrate(M**2, (xi, -1, +1))
print(simplify(U))

2*E*I*(3*W_1**2 - 3*W_1*W_2*h - 6*W_1*W_3 - 3*W_1*W_4*h + W_2**2*h**2 + 3*W_2*W_3*h + W_2*W_4*h**2 + 3*W_3**2 + 3*W_3*W_4*h + W_4**2*h**2)/h**3


Now the partial derivative of the strain energy  $U$ with respect to the first degree of freedom $W_1$ will reveal the work-conjugate generalized force,  namely the shear force that works on the first degree of freedom (vertical displacement at the left hand side end)

In [127]:
print(simplify(diff(U, W_1)))

6*E*I*(2*W_1 - W_2*h - 2*W_3 - W_4*h)/h**3


This is actually the first row of the $4\times4$ stiffness matrix multiplied by the degrees of freedom. The first row of the stiffness matrix can therefore be written as

$$
[12 EI/h^3, -6EI/h^2, 12 EI/h^3, -6EI/h^2]
$$

And similarly the second row follows as

In [128]:
print(simplify(diff(U, W_2)))

2*E*I*(-3*W_1 + 2*W_2*h + 3*W_3 + W_4*h)/h**2


The remaining two rows follow from

In [129]:
print(simplify(diff(U, W_3)))
print(simplify(diff(U, W_4)))

6*E*I*(-2*W_1 + W_2*h + 2*W_3 + W_4*h)/h**3
2*E*I*(-3*W_1 + W_2*h + 3*W_3 + 2*W_4*h)/h**2


The weighted residual method states that the stiffness matrix can be obtained as

$$
K = EI \int_{-1}^{+1} B^T B \; d\xi\; (h/2)
$$

The symbolic code below does precisely this.

In [130]:
K = E * I * integrate(B.T * B, (xi, -1, 1)) * (h/2) 
print(K)

Matrix([[12*E*I/h**3, -6*E*I/h**2, -12*E*I/h**3, -6*E*I/h**2], [-6*E*I/h**2, 4*E*I/h, 6*E*I/h**2, 2*E*I/h], [-12*E*I/h**3, 6*E*I/h**2, 12*E*I/h**3, 6*E*I/h**2], [-6*E*I/h**2, 2*E*I/h, 6*E*I/h**2, 4*E*I/h]])


We can finally show that the same result can be obtained with numerical integration. Here we use two-point Gauss rule, which should be "exact", since the integrand is only a quadratic function of $\xi$. 

In [131]:
xiG = [-1/sqrt(3), 1/sqrt(3)]
WG = [1, 1]
K = zeros(4, 4)
for q in range(2):
    B1 = B.subs(xi, xiG[q])
    K += B1.T * B1 * WG[q] * (h/2)
print(simplify(K))
    

Matrix([[12/h**3, -6/h**2, -12/h**3, -6/h**2], [-6/h**2, 4/h, 6/h**2, 2/h], [-12/h**3, 6/h**2, 12/h**3, 6/h**2], [-6/h**2, 2/h, 6/h**2, 4/h]])
